<a href="https://colab.research.google.com/github/Ganeshlendale/C-A_Internship/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving Final.csv to Final.csv


In [8]:
import pandas as pd

df = pd.read_csv('/content/Final.csv')  # Replace with actual uploaded filename
print(df.head())


   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

   TrafficConditionNearby  QueueLength  IsSpecialDay  ... LastUpdatedTime  \
0                     0.0            1         False  ...           07:59   
1                     0.0            1         False  ...           08:25   
2                     0.0            2         False  ...           08:59   
3                     0.0            2         False  ...           09:32   
4                     0.0            2         False  ...           09:59   

  OccupancyRate         Timestamp Hour  DayOfWeek 

In [9]:
print(df.columns)


Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'OccupancyRate',
       'Timestamp', 'Hour', 'DayOfWeek', 'Load', 'OccupancyRounded',
       'OccupancyRatio', 'Next_Price_01', 'Next_price_02'],
      dtype='object')


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

# Step 1: Load the dataset
df = pd.read_csv('/content/Final.csv')  # Replace with your actual filename

# Step 2: Define features and targets
target_cols = ['Next_Price_01', 'Next_price_02']
X = df.drop(columns=target_cols)
y = df[target_cols]

# Step 3: Clean and preprocess
X = X.fillna(0)
y = y.fillna(0)
X = pd.get_dummies(X)

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 5: Define candidate models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "MLP Regressor (Neural Net)": MLPRegressor(random_state=42, max_iter=1000)
}

# Step 6: Train and evaluate models
results = []

for name, base_model in models.items():
    model = MultiOutputRegressor(base_model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')

    results.append({
        "Model": name,
        "MSE_Price_01": mse[0],
        "MSE_Price_02": mse[1],
        "R2_Price_01": r2[0],
        "R2_Price_02": r2[1]
    })

# Step 7: Compile results
results_df = pd.DataFrame(results)

# Step 8: Select best model(s)
best_by_mse = results_df.sort_values(by=["MSE_Price_01", "MSE_Price_02"]).iloc[0]
best_by_r2 = results_df.sort_values(by=["R2_Price_01", "R2_Price_02"], ascending=False).iloc[0]

print("\n📊 All Model Performance:")
print(results_df.sort_values(by=["MSE_Price_01", "MSE_Price_02"]).to_string(index=False))

print("\n🏆 Best Model (Lowest MSE):", best_by_mse['Model'])
print("  MSE_Price_01:", best_by_mse['MSE_Price_01'])
print("  MSE_Price_02:", best_by_mse['MSE_Price_02'])

print("\n⭐ Best Model (Highest R²):", best_by_r2['Model'])
print("  R2_Price_01:", best_by_r2['R2_Price_01'])
print("  R2_Price_02:", best_by_r2['R2_Price_02'])



📊 All Model Performance:
                     Model  MSE_Price_01  MSE_Price_02  R2_Price_01  R2_Price_02
         Linear Regression  1.543527e-24     29.644367     1.000000     0.263179
             Random Forest  1.297158e-04      2.925525     0.999987     0.927285
         Gradient Boosting  4.752141e-04      2.907925     0.999951     0.927722
                   XGBoost  4.798311e-04      3.219356     0.999951     0.919982
       K-Nearest Neighbors  1.985802e-02      4.189369     0.997966     0.895872
MLP Regressor (Neural Net)  2.797929e-01      5.808998     0.971341     0.855615

🏆 Best Model (Lowest MSE): Linear Regression
  MSE_Price_01: 1.5435273858511003e-24
  MSE_Price_02: 29.64436663339867

⭐ Best Model (Highest R²): Linear Regression
  R2_Price_01: 1.0
  R2_Price_02: 0.2631786712527615


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib  # for saving the model

# Step 1: Load the dataset
df = pd.read_csv('/content/Final.csv')  # Replace with your actual filename

# Step 2: Define targets and features
target_cols = ['Next_Price_01', 'Next_price_02']
X = df.drop(columns=target_cols)
y = df[target_cols]

# Step 3: Handle missing values and categorical features
X = X.fillna(0)
y = y.fillna(0)
X = pd.get_dummies(X)

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 5: Train Random Forest Regressor
rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
rf_model.fit(X_train, y_train)

# Step 6: Predict on test data
y_pred = rf_model.predict(X_test)

# Step 7: Evaluate the model
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

print("✅ Random Forest Model Evaluation:")
print("MSE for Next_Price_01:", mse[0])
print("MSE for Next_price_02:", mse[1])
print("R² for Next_Price_01:", r2[0])
print("R² for Next_price_02:", r2[1])

# Step 8: Save the model to file
joblib.dump(rf_model, 'random_forest_price_model.pkl')
print("\n📦 Model saved as: 'random_forest_price_model.pkl'")


✅ Random Forest Model Evaluation:
MSE for Next_Price_01: 0.00012971579606507847
MSE for Next_price_02: 2.9255253028992168
R² for Next_Price_01: 0.9999867132778347
R² for Next_price_02: 0.9272850228974943

📦 Model saved as: 'random_forest_price_model.pkl'
